create flag parameter


In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print (type(incremental_flag))

<class 'str'>


creating dimenion model
__

#
# fetch relative columns

In [0]:
df_source=spark.sql('''
select DISTINCT(Dealer_ID) AS Dealer_ID,DealerName
from parquet.`abfss://silver@azurecarsalesdata7.dfs.core.windows.net/carsales`
''')

In [0]:
display(df_source)

Dealer_ID DealerName DLR0124 McLaren Motors DLR0095 Isuzu Motors DLR0245 Jennings Ford Automobile Dealership DLR0171 Samsung Motors DLR0052 e.GO Mobile Motors DLR0175 Saturn Motors DLR0263 Auto-Union Motors DLR0080 Hindustan Motors DLR0066 Freightliner Motors DLR0258 null DLR0007 AMC, Eagle Motors DLR0141 Noble Motors DLR0251 Alfa Romeo Motors DLR0038 Cizeta-Moroder Motors DLR0155 Piaggio Motors DLR0055 Facel Vega Motors DLR0049 Dodge Motors DLR0013 Aston Martin Motors DLR0129 Mia Motors DLR0183 Smart Motors DLR0177 Seat Motors DLR0229 Herald Motors DLR0149 Panoz Motors DLR0170 Saleen Motors DLR0213 DKW Motors DLR0077 Heinkel Motors DLR0238 Cass Motor Sales DLR0115 Mahindra Motors DLR0253 Alvis Motors DLR0219 MG Motors DLR0166 Rivian Motors DLR0262 Autobianchi Motors DLR0099 Jensen Motors DLR0159 Porsche Motors DLR0118 Maruti Motors DLR0259 Asia Motors Motors DLR0241 Colman Automotive Building DLR0207 Westfield Motors DLR0163 Ranger Motors DLR0157 Polestar Motors DLR0133 Monica Motors DLR0167 Rolls-Royce Motors DLR0086 Hummer Motors DLR0174 Santana Motors DLR0147 Pagani Motors DLR0193 Tazzari Motors DLR0134 Monteverdi Motors DLR0148 Panhard Motors DLR0084 Hudson Motors DLR0235 Buick Automobile Company Building DLR0037 Citroën Motors DLR0051 Edsel Motors DLR0237 Casa de cadillac DLR0021 Bond Motors DLR0064 Ford Europe Motors DLR0109 Ligier Motors DLR0101 Kia Motors DLR0045 Delahaye Motors DLR0265 Bentley Motors DLR0035 Chrysler Motors DLR0153 Peugeot Motors DLR0019 Bitter Motors DLR0088 IFA (including Trabant, Wartburg, Barkas) Motors DLR0083 Honda India Motors DLR0120 Mastretta Motors DLR0191 Tata Motors DLR0216 Maybach Motors DLR0200 UAZ Motors DLR0186 Standard-Triumph Motors DLR0173 Santa Matilde Motors DLR0103 KTM Motors DLR0121 Matra (including Deutsch-Bonnet, René Bonnet) Motors DLR0131 Mini Motors DLR0122 Mazda Motors DLR0011 Artega Motors DLR0143 Oldsmobile Motors DLR0261 Audi Motors DLR0156 Plymouth Motors DLR0178 Shelby SuperCars Motors DLR0026 Austin, Rover Motors DLR0030 Cadillac Motors DLR0106 Lancia Motors DLR0247 Kindel Building DLR0107 Land Rover Motors DLR0067 FSO Motors DLR0061 Ford America Motors DLR0230 Zion Automobils DLR0032 Checker Motors DLR0201 Vauxhall Motors DLR0014 Audi Motors DLR0256 Ariel Motors DLR0161 Proton Motors DLR0078 Hennessey Motors DLR0063 Ford do Brasil Motors DLR0214 Horch Motors DLR0202 Venturi Motors DLR0240 Clemens Automobile Company Building DLR0195 Tornado Motors DLR0090 null DLR0017 Auverland Motors DLR0211 ZIL Motors DLR0154 PGO Motors DLR0181 Singer Motors DLR0243 Howard Motor Company Building DLR0044 null DLR0254 AMC, Eagle Motors DLR0031 Caterham Motors DLR0142 NSU Motors DLR0136 Moskwitch Motors DLR0012 Asia Motors Motors DLR0053 Eicher Polaris Motors DLR0179 Simca Motors DLR0025 Austin-Healey Motors DLR0197 Toyota India Motors DLR0198 Turner Motors DLR0029 Bytom Motors DLR0039 Dacia Motors DLR0001 AC Cars Motors DLR0043 Daimler Motors DLR0060 Fisker, Karma Motors DLR0125 Mercedes-Benz (including Maybach) Motors DLR0004 Alfa Romeo Motors DLR0130 Micro Motors DLR0249 Acura Motors DLR0028 Buick Motors DLR0231 Atlantic Motor Company DLR0139 Nissan Motors DLR0003 Aixam-Mega (including Arola) Motors DLR0069 Geo Motors DLR0102 Koenigsegg Motors DLR0208 Wiesmann Motors DLR0232 2008 NRHP-listed DLR0205 Volkswagen South Africa Motors DLR0137 Nash Motors DLR0042 Daihatsu Motors DLR0105 Lamborghini Motors DLR0126 Mercury Motors DLR0016 Auto-Union Motors DLR0248 AC Cars Motors DLR0065 Ford India Motors DLR0068 GAZ Motors DLR0151 Peel Motors DLR0192 Tatra Motors DLR0244 Hupmobile Building DLR0072 Glas Motors DLR0009 Ariel Motors DLR0093 Iso Motors DLR0098 Jeep Motors DLR0233 Blankinship Motor Company Building DLR0097 Jaguar Motors DLR0160 Premier Motors DLR0024 British Leyland Motors DLR0085 Humber Motors DLR0036 Chrysler Europe Motors DLR0008 Anadol Motors DLR0204 Volkswagen do Brasil Motors DLR0116 Marcos Motors DLR0146 Packard Motors DLR0182 Skoda Motors DLR0150 Panther Motors DLR0225 

dim_dealer sink -Initial and incremental

In [0]:
if spark.catalog.tableExists('cars_sales_catalog.silver.dim_dealer'):
   df_sink=spark.sql('''
   select dim_dealer_key,Dealer_ID,DealerName
   from cars_sales_catalog.silver.dim_dealer
   ''')


else:
  df_sink=spark.sql('''
  select 1 as  dim_dealer_key,Dealer_ID,DealerName
  from parquet.`abfss://silver@azurecarsalesdata7.dfs.core.windows.net/carsales`
  where 1=0''')#get only schema for initial run



In [0]:
display(df_sink)

dim_dealer_key Dealer_ID DealerName

#existing records


In [0]:
df_filter = df_source.join(df_sink,df_source['Dealer_ID']==df_sink['Dealer_ID'],'left').select(df_source['Dealer_ID'],df_source['DealerName'],df_sink['dim_dealer_key'])#left join source and sink 

In [0]:
display(df_filter)

Dealer_ID DealerName dim_dealer_key DLR0124 McLaren Motors null DLR0095 Isuzu Motors null DLR0245 Jennings Ford Automobile Dealership null DLR0171 Samsung Motors null DLR0052 e.GO Mobile Motors null DLR0175 Saturn Motors null DLR0263 Auto-Union Motors null DLR0080 Hindustan Motors null DLR0066 Freightliner Motors null DLR0258 null null DLR0007 AMC, Eagle Motors null DLR0141 Noble Motors null DLR0251 Alfa Romeo Motors null DLR0038 Cizeta-Moroder Motors null DLR0155 Piaggio Motors null DLR0055 Facel Vega Motors null DLR0049 Dodge Motors null DLR0013 Aston Martin Motors null DLR0129 Mia Motors null DLR0183 Smart Motors null DLR0177 Seat Motors null DLR0229 Herald Motors null DLR0149 Panoz Motors null DLR0170 Saleen Motors null DLR0213 DKW Motors null DLR0077 Heinkel Motors null DLR0238 Cass Motor Sales null DLR0115 Mahindra Motors null DLR0253 Alvis Motors null DLR0219 MG Motors null DLR0166 Rivian Motors null DLR0262 Autobianchi Motors null DLR0099 Jensen Motors null DLR0159 Porsche Motors null DLR0118 Maruti Motors null DLR0259 Asia Motors Motors null DLR0241 Colman Automotive Building null DLR0207 Westfield Motors null DLR0163 Ranger Motors null DLR0157 Polestar Motors null DLR0133 Monica Motors null DLR0167 Rolls-Royce Motors null DLR0086 Hummer Motors null DLR0174 Santana Motors null DLR0147 Pagani Motors null DLR0193 Tazzari Motors null DLR0134 Monteverdi Motors null DLR0148 Panhard Motors null DLR0084 Hudson Motors null DLR0235 Buick Automobile Company Building null DLR0037 Citroën Motors null DLR0051 Edsel Motors null DLR0237 Casa de cadillac null DLR0021 Bond Motors null DLR0064 Ford Europe Motors null DLR0109 Ligier Motors null DLR0101 Kia Motors null DLR0045 Delahaye Motors null DLR0265 Bentley Motors null DLR0035 Chrysler Motors null DLR0153 Peugeot Motors null DLR0019 Bitter Motors null DLR0088 IFA (including Trabant, Wartburg, Barkas) Motors null DLR0083 Honda India Motors null DLR0120 Mastretta Motors null DLR0191 Tata Motors null DLR0216 Maybach Motors null DLR0200 UAZ Motors null DLR0186 Standard-Triumph Motors null DLR0173 Santa Matilde Motors null DLR0103 KTM Motors null DLR0121 Matra (including Deutsch-Bonnet, René Bonnet) Motors null DLR0131 Mini Motors null DLR0122 Mazda Motors null DLR0011 Artega Motors null DLR0143 Oldsmobile Motors null DLR0261 Audi Motors null DLR0156 Plymouth Motors null DLR0178 Shelby SuperCars Motors null DLR0026 Austin, Rover Motors null DLR0030 Cadillac Motors null DLR0106 Lancia Motors null DLR0247 Kindel Building null DLR0107 Land Rover Motors null DLR0067 FSO Motors null DLR0061 Ford America Motors null DLR0230 Zion Automobils null DLR0032 Checker Motors null DLR0201 Vauxhall Motors null DLR0014 Audi Motors null DLR0256 Ariel Motors null DLR0161 Proton Motors null DLR0078 Hennessey Motors null DLR0063 Ford do Brasil Motors null DLR0214 Horch Motors null DLR0202 Venturi Motors null DLR0240 Clemens Automobile Company Building null DLR0195 Tornado Motors null DLR0090 null null DLR0017 Auverland Motors null DLR0211 ZIL Motors null DLR0154 PGO Motors null DLR0181 Singer Motors null DLR0243 Howard Motor Company Building null DLR0044 null null DLR0254 AMC, Eagle Motors null DLR0031 Caterham Motors null DLR0142 NSU Motors null DLR0136 Moskwitch Motors null DLR0012 Asia Motors Motors null DLR0053 Eicher Polaris Motors null DLR0179 Simca Motors null DLR0025 Austin-Healey Motors null DLR0197 Toyota India Motors null DLR0198 Turner Motors null DLR0029 Bytom Motors null DLR0039 Dacia Motors null DLR0001 AC Cars Motors null DLR0043 Daimler Motors null DLR0060 Fisker, Karma Motors null DLR0125 Mercedes-Benz (including Maybach) Motors null DLR0004 Alfa Romeo Motors null DLR0130 Micro Motors null DLR0249 Acura Motors null DLR0028 Buick Motors null DLR0231 Atlantic Motor Company null DLR0139 Nissan Motors null DLR0003 Aixam-Mega (including Arola) Motors null DLR0069 Geo Motors null DLR0102 Koenigsegg Motors null DLR0208 Wiesmann Motors null DLR0232 2008 NRHP-listed null DLR0205 Volkswagen South Af

# Filtering new records and old records

In [0]:
df_filter_old= df_filter.filter(col('dim_dealer_key').isNotNull())

In [0]:
display(df_filter_old)#all are null

Dealer_ID DealerName dim_dealer_key

In [0]:
df_filter_new= df_filter.filter(col('dim_dealer_key').isNull()).select(df_source['Dealer_ID'],df_source['DealerName'])#get dealer id,dealer name from where dim dealer key is null

In [0]:
display(df_filter_new)

Dealer_ID DealerName DLR0124 McLaren Motors DLR0095 Isuzu Motors DLR0245 Jennings Ford Automobile Dealership DLR0171 Samsung Motors DLR0052 e.GO Mobile Motors DLR0175 Saturn Motors DLR0263 Auto-Union Motors DLR0080 Hindustan Motors DLR0066 Freightliner Motors DLR0258 null DLR0007 AMC, Eagle Motors DLR0141 Noble Motors DLR0251 Alfa Romeo Motors DLR0038 Cizeta-Moroder Motors DLR0155 Piaggio Motors DLR0055 Facel Vega Motors DLR0049 Dodge Motors DLR0013 Aston Martin Motors DLR0129 Mia Motors DLR0183 Smart Motors DLR0177 Seat Motors DLR0229 Herald Motors DLR0149 Panoz Motors DLR0170 Saleen Motors DLR0213 DKW Motors DLR0077 Heinkel Motors DLR0238 Cass Motor Sales DLR0115 Mahindra Motors DLR0253 Alvis Motors DLR0219 MG Motors DLR0166 Rivian Motors DLR0262 Autobianchi Motors DLR0099 Jensen Motors DLR0159 Porsche Motors DLR0118 Maruti Motors DLR0259 Asia Motors Motors DLR0241 Colman Automotive Building DLR0207 Westfield Motors DLR0163 Ranger Motors DLR0157 Polestar Motors DLR0133 Monica Motors DLR0167 Rolls-Royce Motors DLR0086 Hummer Motors DLR0174 Santana Motors DLR0147 Pagani Motors DLR0193 Tazzari Motors DLR0134 Monteverdi Motors DLR0148 Panhard Motors DLR0084 Hudson Motors DLR0235 Buick Automobile Company Building DLR0037 Citroën Motors DLR0051 Edsel Motors DLR0237 Casa de cadillac DLR0021 Bond Motors DLR0064 Ford Europe Motors DLR0109 Ligier Motors DLR0101 Kia Motors DLR0045 Delahaye Motors DLR0265 Bentley Motors DLR0035 Chrysler Motors DLR0153 Peugeot Motors DLR0019 Bitter Motors DLR0088 IFA (including Trabant, Wartburg, Barkas) Motors DLR0083 Honda India Motors DLR0120 Mastretta Motors DLR0191 Tata Motors DLR0216 Maybach Motors DLR0200 UAZ Motors DLR0186 Standard-Triumph Motors DLR0173 Santa Matilde Motors DLR0103 KTM Motors DLR0121 Matra (including Deutsch-Bonnet, René Bonnet) Motors DLR0131 Mini Motors DLR0122 Mazda Motors DLR0011 Artega Motors DLR0143 Oldsmobile Motors DLR0261 Audi Motors DLR0156 Plymouth Motors DLR0178 Shelby SuperCars Motors DLR0026 Austin, Rover Motors DLR0030 Cadillac Motors DLR0106 Lancia Motors DLR0247 Kindel Building DLR0107 Land Rover Motors DLR0067 FSO Motors DLR0061 Ford America Motors DLR0230 Zion Automobils DLR0032 Checker Motors DLR0201 Vauxhall Motors DLR0014 Audi Motors DLR0256 Ariel Motors DLR0161 Proton Motors DLR0078 Hennessey Motors DLR0063 Ford do Brasil Motors DLR0214 Horch Motors DLR0202 Venturi Motors DLR0240 Clemens Automobile Company Building DLR0195 Tornado Motors DLR0090 null DLR0017 Auverland Motors DLR0211 ZIL Motors DLR0154 PGO Motors DLR0181 Singer Motors DLR0243 Howard Motor Company Building DLR0044 null DLR0254 AMC, Eagle Motors DLR0031 Caterham Motors DLR0142 NSU Motors DLR0136 Moskwitch Motors DLR0012 Asia Motors Motors DLR0053 Eicher Polaris Motors DLR0179 Simca Motors DLR0025 Austin-Healey Motors DLR0197 Toyota India Motors DLR0198 Turner Motors DLR0029 Bytom Motors DLR0039 Dacia Motors DLR0001 AC Cars Motors DLR0043 Daimler Motors DLR0060 Fisker, Karma Motors DLR0125 Mercedes-Benz (including Maybach) Motors DLR0004 Alfa Romeo Motors DLR0130 Micro Motors DLR0249 Acura Motors DLR0028 Buick Motors DLR0231 Atlantic Motor Company DLR0139 Nissan Motors DLR0003 Aixam-Mega (including Arola) Motors DLR0069 Geo Motors DLR0102 Koenigsegg Motors DLR0208 Wiesmann Motors DLR0232 2008 NRHP-listed DLR0205 Volkswagen South Africa Motors DLR0137 Nash Motors DLR0042 Daihatsu Motors DLR0105 Lamborghini Motors DLR0126 Mercury Motors DLR0016 Auto-Union Motors DLR0248 AC Cars Motors DLR0065 Ford India Motors DLR0068 GAZ Motors DLR0151 Peel Motors DLR0192 Tatra Motors DLR0244 Hupmobile Building DLR0072 Glas Motors DLR0009 Ariel Motors DLR0093 Iso Motors DLR0098 Jeep Motors DLR0233 Blankinship Motor Company Building DLR0097 Jaguar Motors DLR0160 Premier Motors DLR0024 British Leyland Motors DLR0085 Humber Motors DLR0036 Chrysler Europe Motors DLR0008 Anadol Motors DLR0204 Volkswagen do Brasil Motors DLR0116 Marcos Motors DLR0146 Packard Motors DLR0182 Skoda Motors DLR0150 Panther Motors DLR0225 

#create surrogate key

#fetch the max surrogate key from table

In [0]:
if (incremental_flag=='0'):
    max_value=1#initial load
else:
    max_value_df=spark.sql("select max(dim_dealer_key) from cars_sales_catalog.silver.dim_dealer")#list
    max_value=max_value_df.collect()[0][0]+1#select the value from the list

#create surrogate key column and add max surrrogate key

In [0]:
df_filter_new=df_filter_new.withColumn('dim_dealer_key',max_value+monotonically_increasing_id())#create a column dim_dealer_key which increments the max value

In [0]:
display(df_filter_new)

Dealer_ID DealerName dim_dealer_key DLR0124 McLaren Motors 1 DLR0095 Isuzu Motors 2 DLR0245 Jennings Ford Automobile Dealership 3 DLR0171 Samsung Motors 4 DLR0052 e.GO Mobile Motors 5 DLR0175 Saturn Motors 6 DLR0263 Auto-Union Motors 7 DLR0080 Hindustan Motors 8 DLR0066 Freightliner Motors 9 DLR0258 null 10 DLR0007 AMC, Eagle Motors 11 DLR0141 Noble Motors 12 DLR0251 Alfa Romeo Motors 13 DLR0038 Cizeta-Moroder Motors 14 DLR0155 Piaggio Motors 15 DLR0055 Facel Vega Motors 16 DLR0049 Dodge Motors 17 DLR0013 Aston Martin Motors 18 DLR0129 Mia Motors 19 DLR0183 Smart Motors 20 DLR0177 Seat Motors 21 DLR0229 Herald Motors 22 DLR0149 Panoz Motors 23 DLR0170 Saleen Motors 24 DLR0213 DKW Motors 25 DLR0077 Heinkel Motors 26 DLR0238 Cass Motor Sales 27 DLR0115 Mahindra Motors 28 DLR0253 Alvis Motors 29 DLR0219 MG Motors 30 DLR0166 Rivian Motors 31 DLR0262 Autobianchi Motors 32 DLR0099 Jensen Motors 33 DLR0159 Porsche Motors 34 DLR0118 Maruti Motors 35 DLR0259 Asia Motors Motors 36 DLR0241 Colman Automotive Building 37 DLR0207 Westfield Motors 38 DLR0163 Ranger Motors 39 DLR0157 Polestar Motors 40 DLR0133 Monica Motors 41 DLR0167 Rolls-Royce Motors 42 DLR0086 Hummer Motors 43 DLR0174 Santana Motors 44 DLR0147 Pagani Motors 45 DLR0193 Tazzari Motors 46 DLR0134 Monteverdi Motors 47 DLR0148 Panhard Motors 48 DLR0084 Hudson Motors 49 DLR0235 Buick Automobile Company Building 50 DLR0037 Citroën Motors 51 DLR0051 Edsel Motors 52 DLR0237 Casa de cadillac 53 DLR0021 Bond Motors 54 DLR0064 Ford Europe Motors 55 DLR0109 Ligier Motors 56 DLR0101 Kia Motors 57 DLR0045 Delahaye Motors 58 DLR0265 Bentley Motors 59 DLR0035 Chrysler Motors 60 DLR0153 Peugeot Motors 61 DLR0019 Bitter Motors 62 DLR0088 IFA (including Trabant, Wartburg, Barkas) Motors 63 DLR0083 Honda India Motors 64 DLR0120 Mastretta Motors 65 DLR0191 Tata Motors 66 DLR0216 Maybach Motors 67 DLR0200 UAZ Motors 68 DLR0186 Standard-Triumph Motors 69 DLR0173 Santa Matilde Motors 70 DLR0103 KTM Motors 71 DLR0121 Matra (including Deutsch-Bonnet, René Bonnet) Motors 72 DLR0131 Mini Motors 73 DLR0122 Mazda Motors 74 DLR0011 Artega Motors 75 DLR0143 Oldsmobile Motors 76 DLR0261 Audi Motors 77 DLR0156 Plymouth Motors 78 DLR0178 Shelby SuperCars Motors 79 DLR0026 Austin, Rover Motors 80 DLR0030 Cadillac Motors 81 DLR0106 Lancia Motors 82 DLR0247 Kindel Building 83 DLR0107 Land Rover Motors 84 DLR0067 FSO Motors 85 DLR0061 Ford America Motors 86 DLR0230 Zion Automobils 87 DLR0032 Checker Motors 88 DLR0201 Vauxhall Motors 89 DLR0014 Audi Motors 90 DLR0256 Ariel Motors 91 DLR0161 Proton Motors 92 DLR0078 Hennessey Motors 93 DLR0063 Ford do Brasil Motors 94 DLR0214 Horch Motors 95 DLR0202 Venturi Motors 96 DLR0240 Clemens Automobile Company Building 97 DLR0195 Tornado Motors 98 DLR0090 null 99 DLR0017 Auverland Motors 100 DLR0211 ZIL Motors 101 DLR0154 PGO Motors 102 DLR0181 Singer Motors 103 DLR0243 Howard Motor Company Building 104 DLR0044 null 105 DLR0254 AMC, Eagle Motors 106 DLR0031 Caterham Motors 107 DLR0142 NSU Motors 108 DLR0136 Moskwitch Motors 109 DLR0012 Asia Motors Motors 110 DLR0053 Eicher Polaris Motors 111 DLR0179 Simca Motors 112 DLR0025 Austin-Healey Motors 113 DLR0197 Toyota India Motors 114 DLR0198 Turner Motors 115 DLR0029 Bytom Motors 116 DLR0039 Dacia Motors 117 DLR0001 AC Cars Motors 118 DLR0043 Daimler Motors 119 DLR0060 Fisker, Karma Motors 120 DLR0125 Mercedes-Benz (including Maybach) Motors 121 DLR0004 Alfa Romeo Motors 122 DLR0130 Micro Motors 123 DLR0249 Acura Motors 124 DLR0028 Buick Motors 125 DLR0231 Atlantic Motor Company 126 DLR0139 Nissan Motors 127 DLR0003 Aixam-Mega (including Arola) Motors 128 DLR0069 Geo Motors 129 DLR0102 Koenigsegg Motors 130 DLR0208 Wiesmann Motors 131 DLR0232 2008 NRHP-listed 132 DLR0205 Volkswagen South Africa Motors 133 DLR0137 Nash Motors 134 DLR0042 Daihatsu Motors 135 DLR0105 Lamborghini Motors 136 DLR0126 Mercury Motors 137 DLR0016 Auto-Union Motors 138 DLR0248 AC Cars Motors 139 DLR0065 Ford India Motors 140 DLR0068 GAZ Motors 141 DLR0

#create final df

In [0]:
df_final=df_filter_new.union(df_filter_old)

In [0]:
display(df_final)

Dealer_ID DealerName dim_dealer_key DLR0124 McLaren Motors 1 DLR0095 Isuzu Motors 2 DLR0245 Jennings Ford Automobile Dealership 3 DLR0171 Samsung Motors 4 DLR0052 e.GO Mobile Motors 5 DLR0175 Saturn Motors 6 DLR0263 Auto-Union Motors 7 DLR0080 Hindustan Motors 8 DLR0066 Freightliner Motors 9 DLR0258 null 10 DLR0007 AMC, Eagle Motors 11 DLR0141 Noble Motors 12 DLR0251 Alfa Romeo Motors 13 DLR0038 Cizeta-Moroder Motors 14 DLR0155 Piaggio Motors 15 DLR0055 Facel Vega Motors 16 DLR0049 Dodge Motors 17 DLR0013 Aston Martin Motors 18 DLR0129 Mia Motors 19 DLR0183 Smart Motors 20 DLR0177 Seat Motors 21 DLR0229 Herald Motors 22 DLR0149 Panoz Motors 23 DLR0170 Saleen Motors 24 DLR0213 DKW Motors 25 DLR0077 Heinkel Motors 26 DLR0238 Cass Motor Sales 27 DLR0115 Mahindra Motors 28 DLR0253 Alvis Motors 29 DLR0219 MG Motors 30 DLR0166 Rivian Motors 31 DLR0262 Autobianchi Motors 32 DLR0099 Jensen Motors 33 DLR0159 Porsche Motors 34 DLR0118 Maruti Motors 35 DLR0259 Asia Motors Motors 36 DLR0241 Colman Automotive Building 37 DLR0207 Westfield Motors 38 DLR0163 Ranger Motors 39 DLR0157 Polestar Motors 40 DLR0133 Monica Motors 41 DLR0167 Rolls-Royce Motors 42 DLR0086 Hummer Motors 43 DLR0174 Santana Motors 44 DLR0147 Pagani Motors 45 DLR0193 Tazzari Motors 46 DLR0134 Monteverdi Motors 47 DLR0148 Panhard Motors 48 DLR0084 Hudson Motors 49 DLR0235 Buick Automobile Company Building 50 DLR0037 Citroën Motors 51 DLR0051 Edsel Motors 52 DLR0237 Casa de cadillac 53 DLR0021 Bond Motors 54 DLR0064 Ford Europe Motors 55 DLR0109 Ligier Motors 56 DLR0101 Kia Motors 57 DLR0045 Delahaye Motors 58 DLR0265 Bentley Motors 59 DLR0035 Chrysler Motors 60 DLR0153 Peugeot Motors 61 DLR0019 Bitter Motors 62 DLR0088 IFA (including Trabant, Wartburg, Barkas) Motors 63 DLR0083 Honda India Motors 64 DLR0120 Mastretta Motors 65 DLR0191 Tata Motors 66 DLR0216 Maybach Motors 67 DLR0200 UAZ Motors 68 DLR0186 Standard-Triumph Motors 69 DLR0173 Santa Matilde Motors 70 DLR0103 KTM Motors 71 DLR0121 Matra (including Deutsch-Bonnet, René Bonnet) Motors 72 DLR0131 Mini Motors 73 DLR0122 Mazda Motors 74 DLR0011 Artega Motors 75 DLR0143 Oldsmobile Motors 76 DLR0261 Audi Motors 77 DLR0156 Plymouth Motors 78 DLR0178 Shelby SuperCars Motors 79 DLR0026 Austin, Rover Motors 80 DLR0030 Cadillac Motors 81 DLR0106 Lancia Motors 82 DLR0247 Kindel Building 83 DLR0107 Land Rover Motors 84 DLR0067 FSO Motors 85 DLR0061 Ford America Motors 86 DLR0230 Zion Automobils 87 DLR0032 Checker Motors 88 DLR0201 Vauxhall Motors 89 DLR0014 Audi Motors 90 DLR0256 Ariel Motors 91 DLR0161 Proton Motors 92 DLR0078 Hennessey Motors 93 DLR0063 Ford do Brasil Motors 94 DLR0214 Horch Motors 95 DLR0202 Venturi Motors 96 DLR0240 Clemens Automobile Company Building 97 DLR0195 Tornado Motors 98 DLR0090 null 99 DLR0017 Auverland Motors 100 DLR0211 ZIL Motors 101 DLR0154 PGO Motors 102 DLR0181 Singer Motors 103 DLR0243 Howard Motor Company Building 104 DLR0044 null 105 DLR0254 AMC, Eagle Motors 106 DLR0031 Caterham Motors 107 DLR0142 NSU Motors 108 DLR0136 Moskwitch Motors 109 DLR0012 Asia Motors Motors 110 DLR0053 Eicher Polaris Motors 111 DLR0179 Simca Motors 112 DLR0025 Austin-Healey Motors 113 DLR0197 Toyota India Motors 114 DLR0198 Turner Motors 115 DLR0029 Bytom Motors 116 DLR0039 Dacia Motors 117 DLR0001 AC Cars Motors 118 DLR0043 Daimler Motors 119 DLR0060 Fisker, Karma Motors 120 DLR0125 Mercedes-Benz (including Maybach) Motors 121 DLR0004 Alfa Romeo Motors 122 DLR0130 Micro Motors 123 DLR0249 Acura Motors 124 DLR0028 Buick Motors 125 DLR0231 Atlantic Motor Company 126 DLR0139 Nissan Motors 127 DLR0003 Aixam-Mega (including Arola) Motors 128 DLR0069 Geo Motors 129 DLR0102 Koenigsegg Motors 130 DLR0208 Wiesmann Motors 131 DLR0232 2008 NRHP-listed 132 DLR0205 Volkswagen South Africa Motors 133 DLR0137 Nash Motors 134 DLR0042 Daihatsu Motors 135 DLR0105 Lamborghini Motors 136 DLR0126 Mercury Motors 137 DLR0016 Auto-Union Motors 138 DLR0248 AC Cars Motors 139 DLR0065 Ford India Motors 140 DLR0068 GAZ Motors 141 DLR0

In [0]:
from delta.tables import DeltaTable

#SCD TYPE 1

In [0]:
if spark.catalog.tableExists('cars_sales_catalog.silver.dim_dealer'):
    delta_table=DeltaTable.forPath(spark,'abfss://gold@azurecarsalesdata7.dfs.core.windows.net/dim_dealer')
    delta_table.alias("target").merge(df_final.alias("source"),"target.dim_dealer_key=source.dim_dealer_key")\
                               .whenMatchedUpdateAll()\
                               .whenNotMatchedInsertAll()\
                               .execute()

else:#initial run
    df_final.write.format('delta')\
            .mode('overwrite')\
            .option('path',"abfss://gold@azurecarsalesdata7.dfs.core.windows.net/dim_dealer")\
            .saveAsTable('cars_sales_catalog.silver.dim_dealer')

In [0]:
%sql
select*from cars_sales_catalog.silver.dim_dealer

Dealer_ID DealerName dim_dealer_key DLR0124 McLaren Motors 1 DLR0095 Isuzu Motors 2 DLR0245 Jennings Ford Automobile Dealership 3 DLR0171 Samsung Motors 4 DLR0052 e.GO Mobile Motors 5 DLR0175 Saturn Motors 6 DLR0263 Auto-Union Motors 7 DLR0080 Hindustan Motors 8 DLR0066 Freightliner Motors 9 DLR0258 null 10 DLR0007 AMC, Eagle Motors 11 DLR0141 Noble Motors 12 DLR0251 Alfa Romeo Motors 13 DLR0038 Cizeta-Moroder Motors 14 DLR0155 Piaggio Motors 15 DLR0055 Facel Vega Motors 16 DLR0049 Dodge Motors 17 DLR0013 Aston Martin Motors 18 DLR0129 Mia Motors 19 DLR0183 Smart Motors 20 DLR0177 Seat Motors 21 DLR0229 Herald Motors 22 DLR0149 Panoz Motors 23 DLR0170 Saleen Motors 24 DLR0213 DKW Motors 25 DLR0077 Heinkel Motors 26 DLR0238 Cass Motor Sales 27 DLR0115 Mahindra Motors 28 DLR0253 Alvis Motors 29 DLR0219 MG Motors 30 DLR0166 Rivian Motors 31 DLR0262 Autobianchi Motors 32 DLR0099 Jensen Motors 33 DLR0159 Porsche Motors 34 DLR0118 Maruti Motors 35 DLR0259 Asia Motors Motors 36 DLR0241 Colman Automotive Building 37 DLR0207 Westfield Motors 38 DLR0163 Ranger Motors 39 DLR0157 Polestar Motors 40 DLR0133 Monica Motors 41 DLR0167 Rolls-Royce Motors 42 DLR0086 Hummer Motors 43 DLR0174 Santana Motors 44 DLR0147 Pagani Motors 45 DLR0193 Tazzari Motors 46 DLR0134 Monteverdi Motors 47 DLR0148 Panhard Motors 48 DLR0084 Hudson Motors 49 DLR0235 Buick Automobile Company Building 50 DLR0037 Citroën Motors 51 DLR0051 Edsel Motors 52 DLR0237 Casa de cadillac 53 DLR0021 Bond Motors 54 DLR0064 Ford Europe Motors 55 DLR0109 Ligier Motors 56 DLR0101 Kia Motors 57 DLR0045 Delahaye Motors 58 DLR0265 Bentley Motors 59 DLR0035 Chrysler Motors 60 DLR0153 Peugeot Motors 61 DLR0019 Bitter Motors 62 DLR0088 IFA (including Trabant, Wartburg, Barkas) Motors 63 DLR0083 Honda India Motors 64 DLR0120 Mastretta Motors 65 DLR0191 Tata Motors 66 DLR0216 Maybach Motors 67 DLR0200 UAZ Motors 68 DLR0186 Standard-Triumph Motors 69 DLR0173 Santa Matilde Motors 70 DLR0103 KTM Motors 71 DLR0121 Matra (including Deutsch-Bonnet, René Bonnet) Motors 72 DLR0131 Mini Motors 73 DLR0122 Mazda Motors 74 DLR0011 Artega Motors 75 DLR0143 Oldsmobile Motors 76 DLR0261 Audi Motors 77 DLR0156 Plymouth Motors 78 DLR0178 Shelby SuperCars Motors 79 DLR0026 Austin, Rover Motors 80 DLR0030 Cadillac Motors 81 DLR0106 Lancia Motors 82 DLR0247 Kindel Building 83 DLR0107 Land Rover Motors 84 DLR0067 FSO Motors 85 DLR0061 Ford America Motors 86 DLR0230 Zion Automobils 87 DLR0032 Checker Motors 88 DLR0201 Vauxhall Motors 89 DLR0014 Audi Motors 90 DLR0256 Ariel Motors 91 DLR0161 Proton Motors 92 DLR0078 Hennessey Motors 93 DLR0063 Ford do Brasil Motors 94 DLR0214 Horch Motors 95 DLR0202 Venturi Motors 96 DLR0240 Clemens Automobile Company Building 97 DLR0195 Tornado Motors 98 DLR0090 null 99 DLR0017 Auverland Motors 100 DLR0211 ZIL Motors 101 DLR0154 PGO Motors 102 DLR0181 Singer Motors 103 DLR0243 Howard Motor Company Building 104 DLR0044 null 105 DLR0254 AMC, Eagle Motors 106 DLR0031 Caterham Motors 107 DLR0142 NSU Motors 108 DLR0136 Moskwitch Motors 109 DLR0012 Asia Motors Motors 110 DLR0053 Eicher Polaris Motors 111 DLR0179 Simca Motors 112 DLR0025 Austin-Healey Motors 113 DLR0197 Toyota India Motors 114 DLR0198 Turner Motors 115 DLR0029 Bytom Motors 116 DLR0039 Dacia Motors 117 DLR0001 AC Cars Motors 118 DLR0043 Daimler Motors 119 DLR0060 Fisker, Karma Motors 120 DLR0125 Mercedes-Benz (including Maybach) Motors 121 DLR0004 Alfa Romeo Motors 122 DLR0130 Micro Motors 123 DLR0249 Acura Motors 124 DLR0028 Buick Motors 125 DLR0231 Atlantic Motor Company 126 DLR0139 Nissan Motors 127 DLR0003 Aixam-Mega (including Arola) Motors 128 DLR0069 Geo Motors 129 DLR0102 Koenigsegg Motors 130 DLR0208 Wiesmann Motors 131 DLR0232 2008 NRHP-listed 132 DLR0205 Volkswagen South Africa Motors 133 DLR0137 Nash Motors 134 DLR0042 Daihatsu Motors 135 DLR0105 Lamborghini Motors 136 DLR0126 Mercury Motors 137 DLR0016 Auto-Union Motors 138 DLR0248 AC Cars Motors 139 DLR0065 Ford India Motors 140 DLR0068 GAZ Motors 141 DLR0